In [ ]:
###############################################################################
import aliases
from everest.utilities import textutils
# tuple(textutils.find_all('__getattr__', path='../everest/ptolemaic'))
# textutils.replace_all('Schematic', 'Compound', path='../everest')

In [ ]:
from everest.utilities import pretty as _pretty

from everest.ptolemaic.system import System as _System
from everest.ptolemaic.essence import Essence as _Essence
from everest.ptolemaic.pathget import PathGet as _PathGet

In [ ]:
from everest.ptolemaic.demiurge import Demiurge as _Demiurge

In [ ]:
class Realm(metaclass=_System):

    ...


class Expression(metaclass=_Essence):

    ...


class Operator(metaclass=_Demiurge):


    valence: POS[int]
    args: ARGS
    kwargs: KWARGS

    @classmethod
    def _parameterise_(cls, /, *args, **kwargs):
        params = super()._parameterise_(*args, **kwargs)
        if (valence := params.valence) is not Ellipsis:
            params.valence = int(valence)
        return params

    @classmethod
    def _dispatch_(cls, params, /):
        valence = params.valence
        if valence == 0:
            kls = cls.Nullary
        elif valence == 1:
            kls = cls.Unary
        elif valence is Ellipsis:
            kls = cls.Ennary
        else:
            raise ValueError
        return kls.partial(*params.args, **params.kwargs)


    class _DemiBase_(ptolemaic):


        dest: POS[Realm]

        def __call__(self, /, *args, **kwargs):
            return self.Expression(self, *args, **kwargs)


        class Expression(mroclass(Expression), metaclass=_System):

            operator: POS['..']

            @property
            def dest(self, /):
                return self.operator.dest

            def _pretty_repr_(self, p, cycle, root=None):
                bound = self.__ptolemaic_class__.__signature__.bind_partial()
                bound.arguments.update(self.params)
                args = tuple(arg for arg in bound.args[1:] if arg is not NotImplemented)
                kwargs = {
                    key: val
                    for key, val in bound.kwargs.items()
                    if val is not NotImplemented
                    }
                if root is None:
                    root = self.operator
                _pretty.pretty_argskwargs(
                    (args, kwargs), p, cycle, root=root
                    )


    class Nullary(demiclass):

        ...


    class NonNullary(demiclass):

        source: POS[Realm] = ANCILLARY
        source = prop('self.dest')


    class Unary(demiclass('.NonNullary')):

        class Expression(mroclass):

            argument: POS[Expression]

            @classmethod
            def _parameterise_(cls, arg, /, **kwargs):
                params = super()._parameterise_(*args, **kwargs)
                if not params.argument in params.operator.source:
                    raise ValueError(
                        "Form arguments must belong to their operator's source."
                        )
                return params


    class Ennary(demiclass('.NonNullary')):

        class Expression(mroclass):

            arguments: ARGS[Expression]

            @classmethod
            def _parameterise_(cls, /, *args, **kwargs):
                params = super()._parameterise_(*args, **kwargs)
                if not all(arg in params.operator.source for arg in params.arguments):
                    raise ValueError(
                        "Form arguments must belong to their operator's source."
                        )
                return params

In [ ]:
realm = Realm()
Operator(..., realm)

In [ ]:
class A(metaclass=_System):
    ...
A()

In [ ]:
def _operator_bodymeth_(body, arg0=0, /, *argn, **kwargs):
    ns = body.namespace
    if isinstance(arg0, _PathGet):
        arg0 = arg0(ns)
    if isinstance(arg0, type):
        optyp = arg0
        args = argn
    else:
        if arg0 is Ellipsis:
            optyp = Ennary
            args = (NotImplemented,)
        else:
            if isinstance(arg0, int):
                if argn:
                    raise ValueError(
                        "Cannot pass both int flag and sources "
                        "to operator constructor."
                        )
                args = tuple(NotImplemented for _ in range(arg0))
            else:
                args = (arg0, *argn)
            arity = len(args)
            if arity == 0:
                optyp = Nullary
            elif arity == 1:
                optyp = Unary
            elif arity == 2:
                raise ValueError
            else:
                raise ValueError
    return body['organ'](
        **optyp.__signature__.bind_partial(*args, **kwargs).arguments,
        )(optyp)

# def _constant_bodymeth_(body, /):
#     return body['organ']()(Atom)


class _AlgebraMeta_(_System):

    @classmethod
    def _yield_bodymeths(meta, body, /):
        yield from super()._yield_bodymeths(body)
        yield 'operator', _operator_bodymeth_.__get__(body)
        # yield 'constant', _constant_bodymeth_.__get__(body)


class Algebra(metaclass=_AlgebraMeta_):

    target: POS = ANCILLARY
    target = prop('self')

    @prop
    def isprimary(self, /):
        return self.target is self

    def __contains__(self, other, /):
        if isinstance(other, Operator.Expression):
            return other.target is self.target
        return False


class Expression(metaclass=_Essence):

    ...


class Operator(metaclass=_System):


    source: POS[Algebra] = ANCILLARY
    source = prop('self.target')

    def _construct_(self, /):
        raise RuntimeError(
            "Operators should only be instantiated "
            "as organs of instances of algebras."
            )

    def __call__(self, /, *args, **kwargs):
        return self.Expression(self, *args, **kwargs)

    @property
    def algebra(self, /):
        return self.__corpus__

    @property
    def target(self, /):
        return self.algebra.target


    class Expression(mroclass(Expression), metaclass=_System):

        operator: POS['Operator']

        @prop
        def canonical(self, /):
            return self.algebra.canonise(*self.arguments)

        @property
        def algebra(self, /):
            return self.operator.algebra

        @property
        def target(self, /):
            return self.algebra.target

        def _pretty_repr_(self, p, cycle, root=None):
            bound = self.__ptolemaic_class__.__signature__.bind_partial()
            bound.arguments.update(self.params)
            args = tuple(arg for arg in bound.args[1:] if arg is not NotImplemented)
            kwargs = {
                key: val
                for key, val in bound.kwargs.items()
                if val is not NotImplemented
                }
            if root is None:
                root = self.operator
            _pretty.pretty_argskwargs(
                (args, kwargs), p, cycle, root=root
                )


class Nullary(Operator):

    ...


class Unary(Operator):

    class Expression(mroclass):

        argument: POS[Expression]

        @classmethod
        def _parameterise_(cls, arg, /, **kwargs):
            params = super()._parameterise_(*args, **kwargs)
            if not params.argument in params.operator.source:
                raise ValueError(
                    "Form arguments must belong to their operator's source."
                    )
            return params


class Ennary(Operator):

    class Expression(mroclass):

        arguments: ARGS[Expression]

        @classmethod
        def _parameterise_(cls, /, *args, **kwargs):
            params = super()._parameterise_(*args, **kwargs)
            if not all(arg in params.operator.source for arg in params.arguments):
                raise ValueError(
                    "Form arguments must belong to their operator's source."
                    )
            return params

In [ ]:
class Foo(Algebra):

    zero = operator(0)
    suc = operator(1)
    neg = operator(1)
    add = operator(...)
    rec = operator(1)
    mul = operator(...)

In [ ]:
foo = Foo()

In [ ]:
foo.zero()

In [ ]:
foo.zero.algebra

In [ ]:
foo.suc(foo.zero)

In [ ]:
foo.add(foo.zero, foo.zero)

In [ ]:
foo.neg(foo.add(foo.zero, foo.zero))